In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Vector Search - RAG Demo



## Setup

Before get started with the Vertex AI services, we need to setup the following.

* Install Python SDK
* Environment variables
* Authentication (Colab only)
* Enable APIs
* Set IAM permissions

### Install the Vertex AI SDK

Vertex AI APIs can be accessed with multiple ways including REST API and Python SDK. In this tutorial we will use the SDK.

In [ ]:
%pip install --upgrade --quiet --user google-cloud-aiplatform

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

For Colab users: you will see "Your session crushed" message when restarting the kernel, but it is expected. Continue to the following cells.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. In Colab or Colab Enterprise, you might see an error message that says "Your session crashed for an unknown reason." This is expected. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Environment variables

Sets environment variables. If asked, replace the following `[your-project-id]` with your project ID and run it.

In [1]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

Please set the project ID manually below


In [2]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "uk-con-gcp-sbx-ukat01-012822"  # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

### Authentication (Colab only)

If you are running this notebook on Colab, you will need to run the following cell authentication. This step is not required if you are using Vertex AI Workbench as it is pre-authenticated.

In [3]:
import sys

# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set IAM permissions

Also, we need to add access permissions to the default service account for using those services.

- Go to [the IAM page](https://console.cloud.google.com/iam-admin/) in the Console
- Look for the principal for default compute service account. It should look like: `<project-number>-compute@developer.gserviceaccount.com`
- Click the edit button at right and click `ADD ANOTHER ROLE` to add `Vertex AI User` and `Service Usage Admin` roles to the account.


### Enable APIs

Run the following to enable APIs for Compute Engine and Vertex AI with this Google Cloud project.

In [4]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

## Run a Query with Vector Search

Finally it's ready to use Vector Search. To run a query, you need to get an embedding for a query item. For example, if you like to find products with similar names to "cloudveil women's excursion short", you need to get the embedding for it before running a query. In this tutorial, we will get the embedding from the `product-embs.json` file as follows:

In [5]:
from vertexai.language_models import TextEmbeddingModel
def generate_text_embeddings(sentences) -> list:
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")
    embeddings = model.get_embeddings(sentences)
    vectors = [embedding.values for embedding in embeddings]
    return vectors

With the `product_embs` dict, you can specify a product ID to get an embedding for it.

In [6]:
#query=["Allowed cost of online course"]
#query=["process for applying sick leave"]
query=["savings account with the best interest rate and suitable for someone under 16"]
qry_emb=generate_text_embeddings(query)

### Run a Query

Then, pass the embedding to `find_neighbors` function to find similar product names.

In [8]:
from vertexai.preview.generative_models import GenerativeModel, Part

my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name="4381980447198937088")

# run query
response = my_index_endpoint.find_neighbors(
    deployed_index_id="vs_bank_prod_deployed_12041551", queries=[qry_emb[0]], num_neighbors=10
)

# show the results
# for idx, neighbor in enumerate(response[0]):
#     print(f"{neighbor.distance:.2f} {product_names[neighbor.id]}")
matching_ids = [neighbor.id for sublist in response for neighbor in sublist]

In [12]:
import json
sentence_file_path = "bank_sentences_041224.json"
data = []
with open(sentence_file_path) as file:
  for line in file:
    d = json.loads(line)
    data.append(d)

In [ ]:
def generate_context(ids,data):
  concatenated_names = ''
  for id in ids:
    for entry in data:
      # print(entry)
      if entry['id'] == id:
        concatenated_names += entry['sentence'] + "\n"
  return concatenated_names.strip()

context = generate_context(matching_ids,data)
print(context)

In [14]:
model = GenerativeModel("gemini-pro")
prompt=f"Based on the context delimited in backticks, answer the query. ```{context}``` {query}"

In [ ]:
print(prompt)

In [ ]:
chat = model.start_chat(history=[])
response = chat.send_message(prompt)
print(response.text)

The `find_neighbors` function only takes milliseconds to fetch the similar items even when you have billions of items on the Index, thanks to the ScaNN algorithm. Vector Search also supports [autoscaling](https://cloud.google.com/vertex-ai/docs/vector-search/deploy-index-public#autoscaling) which can automatically resize the number of nodes based on the demands of your workloads.

# IMPORTANT: Cleaning Up

In case you are using your own Cloud project, not a temporary project on Qwiklab, please make sure to delete all the Indexes and Index Endpoints after finishing this tutorial. Otherwise the remaining objects would **incur unexpected costs**.

If you used Workbench, you may also need to delete the Notebooks from [the console](https://console.cloud.google.com/vertex-ai/workbench).

In [ ]:
# wait for a confirmation
input("Press Enter to delete Index Endpoint and Index:")

# delete Index Endpoint
my_index_endpoint.undeploy_all()
my_index_endpoint.delete(force=True)

# delete Index
my_index.delete()

## Utilities

It can take some time to create or deploy indexes, and in that time you might lose connection with the Colab runtime. If you lose connection, instead of creating or deploying your new index again, you can check [the Vector Search Console](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints) and use the existing ones to continue.


### Get an existing Index

To get an index object that already exists, replace the following `[numeric-index-id]` with the index ID and run the cell. You can check the ID on [the Vector Search Console > INDEXES tab](https://console.cloud.google.com/vertex-ai/matching-engine/indexes).

In [ ]:
my_index_id = "8619498260647641088"  # @param {type:"string"}
my_index = aiplatform.MatchingEngineIndex(my_index_id)

### Get an existing Index Endpoint

To get an index endpoint object that already exists, replace the following `[numeric-index-endpoint-id]` with the index endpoint ID and run the cell. You can check the ID on [the Vector Search Console > INDEX ENDPOINTS tab](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints).

In [ ]:
my_index_endpoint_id = "4381980447198937088"  # @param {type:"string"}
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint_id)